In [44]:
from application.trading.exchange import Exchange
from application.trading.kraken_futures_api import FuturesApi
from application.trading.kraken_api import KrakenApi
import pandas as pd
import pandas_ta as ta
import numpy as np

import urllib.request as urllib2
import urllib.parse as urllib
import json
import importlib

In [2]:
path = "https://futures.kraken.com/"
public = "3EWPFVk88Cmol8hw8falnfA67Q5RAhnT8jysgfif9gnKmhf0KacIddHA"
private = "D06lgIZD5y/9z6bZL17S7SioS0zwEEQGQH0T7+kVfto1CePZbvzR7UQUoKZzBT/uWEyVy/vmwislBLfCc9/a7Ip2"
futApi = FuturesApi(path, public, private)
import requests
# data1 = requests.get('https://api.kraken.com/0/public/Assets').json()
# data1['result']

In [6]:
pair = "ETHUSD" #"XBTUSD"

# Enum: 1 5 15 30 60 240 1440 10080 21600 | Time frame interval in minutes Default: 1
interval = 60

# Return up to 720 OHLC data points since given timestamp | Example: since=1548111600
since = 1548111600
exch = Exchange()
df_org = exch.ohlc(pair, interval)

entry: breaking out of tight bollinger band in the direction of the trend

In [179]:
df = df_org.copy()
# BBands
std = 1.0
length = 55 # 15
mamode="sma" #"ema"
offset = 0

fastMaLength = 20
slowMaLength = 50
maKind = "ema"

fastMa = f"{maKind.upper()}_{fastMaLength}"
slowMa = f"{maKind.upper()}_{slowMaLength}"

bbu = f"BBU_{length}_{std}"
bbm = f"BBM_{length}_{std}"
bbl = f"BBL_{length}_{std}"
bbb = f"BBB_{length}_{std}" # Bandwidth
bbp = f"BBP_{length}_{std}" # Percent

MyStrategy = ta.Strategy(
    name="strategy_3",
    ta=[{
        "close": 'close', 
        "kind": "bbands", 
        "length": length,
        "std": std,
        "mamode": mamode,
        "ddof": offset,
    },
    {
        "close": 'close',
        "kind": maKind, 
        "length": fastMaLength
    },
    {
        "close": 'close',
        "kind": maKind, 
        "length": slowMaLength
    },]
)
df.ta.strategy(MyStrategy)
df["up_trend"] = df["EMA_20"] > df["EMA_50"]
df["ema_cross_up"] = (df["EMA_20"] > df["EMA_50"]) & (df["EMA_20"].shift() < df["EMA_50"].shift())
df["ema_cross_down"] = (df["EMA_20"] < df["EMA_50"]) & (df["EMA_20"].shift() > df["EMA_50"].shift())
df["bb_breakout"] = np.where(df['close'] > df[bbu].shift(), True, False)
df["bb_signal"] = (df['bb_breakout'] == True) & (df['bb_breakout'].shift() == False)
df["entry_signal"] = (df["up_trend"] == True) & (df["bb_signal"] == True)
df.tail()

,open,high,low,close,vwap,volume,count,BBL_55_1.0,BBM_55_1.0,BBU_55_1.0,BBB_55_1.0,BBP_55_1.0,EMA_20,EMA_50,up_trend,ema_cross_up,ema_cross_down,bb_breakout,bb_signal,entry_signal
time,,,,,,,,,,,,,,,,,,,,
2023-07-08 12:00:00,1859.33,1859.80,1856.42,1856.42,1858.45,60.380981,129,1850.893542,1874.717091,1898.540640,2.541562,0.115987,1863.904621,1873.811113,False,False,False,False,False,False
2023-07-08 13:00:00,1856.73,1861.17,1856.73,1859.89,1858.87,43.924367,127,1850.602542,1873.562727,1896.522913,2.450965,0.202251,1863.522276,1873.265187,False,False,False,False,False,False
2023-07-08 14:00:00,1860.13,1861.70,1859.39,1861.10,1860.50,58.300558,168,1850.476410,1872.403273,1894.330135,2.342109,0.242251,1863.291583,1872.788121,False,False,False,False,False,False
2023-07-08 15:00:00,1861.57,1862.44,1860.13,1860.13,1861.43,27.363535,146,1851.442134,1870.787455,1890.132775,2.068147,0.224547,1862.990480,1872.291724,False,False,False,False,False,False
2023-07-08 16:00:00,1860.13,1861.93,1860.09,1861.58,1860.46,1.593489,17,1852.465697,1869.350727,1886.235757,1.806513,0.269893,1862.856148,1871.871656,False,False,False,False,False,False


In [138]:
import plotly.graph_objects as go
import plotly.express as px

ohlc = df.copy()
# ohlc['20 SMA'] = ohlc['close'].rolling(20).mean()
# df_filtered = df[df['bb_signal']]
df_filtered = df[df['entry_signal']]


fig = go.Figure(data=[go.Candlestick(x = ohlc.index,
                                    open = ohlc['open'],
                                    high = ohlc['high'],
                                    low = ohlc['low'],
                                    close = ohlc['close'],
                                    ),
                     go.Scatter(x=ohlc.index, y=ohlc[bbu], line=dict(color='purple', width=1), name="BBU"),
                    #  go.Scatter(x=ohlc.index, y=ohlc[bbm], line=dict(color='purple', width=1), name="BBM"),
                    #  go.Scatter(x=ohlc.index, y=ohlc[bbl], line=dict(color='purple', width=1), name="BBL"),
                     go.Scatter(x=ohlc.index, y=ohlc[fastMa], line=dict(color='blue', width=1), name="Fast MA"),
                     go.Scatter(x=ohlc.index, y=ohlc[slowMa], line=dict(color='orange', width=1), name="Slow MA"),
                    #  go.Scatter(x=ohlc.index, y=ohlc["bb_breakout"], line=dict(color='blue', width=4), name="BB Breakout"),
                     # go.scatter.Marker(x=ohlc.index, y=ohlc[bbl])
                         
                        #  x=ohlc.index, y=ohlc[bbb], ,  line=dict(color='purple', width=1), name="BBB"),
                     ])

fig.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['close'],
    mode='markers',
    marker=dict(
        color= 'blue', # df_filtered['entry_signal'],
        # colorscale='Viridis',
        showscale=False,
        symbol='circle'
    )
))

fig.show()

In [132]:
def backtest(dfB: pd.DataFrame):

    
    # dfB = dfB[dfB['entry_signal']]
    dfB["entry_price"] = dfB['close']
    dfB["exit_price"] = dfB['close'].shift(-1)
    dfB["pnl"] = dfB['exit_price'] - dfB['entry_price']
    dfB["pnl_pct"] = dfB['pnl'] / dfB['entry_price']
    dfB["cum_pnl"] = dfB['pnl_pct'].cumsum()
    return dfB

11